# Python exceptions

## The try statement

### Antipatterns and "code smells"

#### References

The best way to structure your error-handling code is a long-term question in computer science and software engineering.  Here's just one casually-located example that hits on many of the major themes in more detail than I present here:

https://medium.com/swlh/return-early-pattern-3d18a41bba8

#### Long indented blocks

If you've got a long process that you want to skip if an exception is thrown it can seem natural to include the entire process inside a `try` clause.  If an exception happens then the rest of the associated block will be skipped, and the relevant `except` clause will run.  The pattern looks a bit like this:

```python
for article_url in articles_to_check:
    try:
        # Here's some code that might fail
        fetch_article(article_url)
        do(article_url)
        some(article_url)
        multi(article_url)
        step(article_url)
        process(article_url)
    except:
        print(f"could not fetch {article_url})
```

This does work, but the indented block can become very long.  Especially when code becomes deeply nested this can make it hard to read and reason about.  Let's set up a working example of the problem and then try some alternative solutions.

In the code snippet below I use several elipses (...), which are no-ops in Python, to stand in for a multi-step process.

In [33]:
def consider(test_value):
    if test_value % 7 == 0:
        raise ValueError(f"Don't like: {test_value}")
    else:
        return f"Happy to find {test_value}"

In [34]:
sum_of_good_values = 0
for x in range(1, 11):
    try:
        print(consider(x))
        ...
        ...
        ...
        ...
        ...
        sum_of_good_values += x
    except ValueError as e:
        print(e)
    print("Still inside the loop after the try statement")
print(f"{sum_of_good_values=}")

Happy to find 1
Still inside the loop after the try statement
Happy to find 2
Still inside the loop after the try statement
Happy to find 3
Still inside the loop after the try statement
Happy to find 4
Still inside the loop after the try statement
Happy to find 5
Still inside the loop after the try statement
Happy to find 6
Still inside the loop after the try statement
Don't like: 7
Still inside the loop after the try statement
Happy to find 8
Still inside the loop after the try statement
Happy to find 9
Still inside the loop after the try statement
Happy to find 10
Still inside the loop after the try statement
sum_of_good_values=48


<div class="alert alert-block alert-warning">
<b>Warning:</b> Read the above output carefully! <br/>Notice that the code inside the loop but after the try statement always runs.
</div>

The problem with this pattern is that the `try` clause needs to include not just the code that might raise an excception, but all the code that shoudln't run if it does.  So, how can we avoid this?

#### Alternatives

In [35]:
sum_of_good_values = 0
for x in range(1, 11):
    try:
        print(consider(x))
    except ValueError as e:
        print(e)
        continue
    ...
    ...
    ...
    ...
    ...
    sum_of_good_values += x
    print("Still inside the loop after the try statement")
print(f"{sum_of_good_values=}")

Happy to find 1
Still inside the loop after the try statement
Happy to find 2
Still inside the loop after the try statement
Happy to find 3
Still inside the loop after the try statement
Happy to find 4
Still inside the loop after the try statement
Happy to find 5
Still inside the loop after the try statement
Happy to find 6
Still inside the loop after the try statement
Don't like: 7
Happy to find 8
Still inside the loop after the try statement
Happy to find 9
Still inside the loop after the try statement
Happy to find 10
Still inside the loop after the try statement
sum_of_good_values=48


Note that the code after the try statement doesn't run if there's an exception!  The `continue` statement skips the remainder of the loop body and starts at the top with the next value.  These approaches aren't strictly and always equivilent.  